In [1]:
import numpy as np
import cv2
import imutils
from skimage.filters import threshold_local

def order_points(pts):
    rect = np.zeros((4,2),dtype="float32")
    s = np.sum(pts,axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts,axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl,tr,br,bl) = rect
    
    widthA = np.sqrt(((br[0]-bl[0])**2)+((br[1]-bl[1])**2))
    widthB = np.sqrt(((tr[0]-tl[0])**2)+((tr[1]-tl[1])**2))
    maxWidth = max(int(widthA),int(widthB))
    
    heightA = np.sqrt(((tr[0]-br[0])**2)+((tr[1]-br[1])**2))
    heightB = np.sqrt(((tl[0]-bl[0])**2)+((tl[1]-bl[1])**2))
    maxHeight = max(int(heightA),int(heightB))
                      
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0,maxHeight-1]],dtype = "float32")
    
    M = cv2.getPerspectiveTransform(rect,dst)
    warped = cv2.warpPerspective(image,M,(maxWidth,maxHeight))
    
    return warped

# TAKE PICTURE

In [2]:
cap = cv2.VideoCapture(0)
while(True):
    ret,image = cap.read()
    show = cv2.flip(image,1)
    cv2.imshow("press enter to click",show)
    if(cv2.waitKey(1) == 13):
        break
cap.release()
cv2.destroyAllWindows()

# LOAD IMAGE

In [4]:
image = cv2.imread("scan.jpg")
cv2.imshow("img",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# EDGE DETECTION 

In [5]:
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image,height=500)

gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
edged = cv2.Canny(blur,10,100)
cv2.imshow("blur", blur)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# CONTOUR DETECTION

In [6]:
contours = cv2.findContours(edged.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse=True)[:5]
screenCnt = None
for c in contours:
    peri = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.05*peri,True)
    a = image.copy()
    cv2.drawContours(a,[approx],0,(0),2)
    cv2.imshow("a",a)
    cv2.waitKey(0)
    if(len(approx) == 4):
        screenCnt = approx
        break
cv2.destroyAllWindows()
cv2.drawContours(a,[screenCnt],-1,(0,255,0),2)
cv2.imshow("a",a)
cv2.waitKey(0)
cv2.destroyAllWindows()

# APPLY PERSPECTIVE TRANSFORM AND THRESHOLD

In [7]:
warped = four_point_transform(orig,screenCnt.reshape(4,2) * ratio)
cv2.imshow("warped",warped)
warped_gray = cv2.cvtColor(warped,cv2.COLOR_BGR2GRAY)
#T = threshold_local(warped_gray,11,offset=10,method="gaussian")
#scanned = (warped_gray > T).astype("uint8")*255

ret,thresh1 = cv2.threshold(warped_gray,195,255,cv2.THRESH_BINARY)
thresh1_blur = cv2.GaussianBlur(thresh1,(3,3),0)
scanned = cv2.adaptiveThreshold(thresh1_blur,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,3,5)

cv2.imshow("original",imutils.resize(orig,height = 650))
cv2.imshow("scanned",imutils.resize(scanned,height = 650))
cv2.waitKey(0)
cv2.destroyAllWindows()